# **Sprawozdanie**

**Autorzy:**
- Krzysztof Maciejewski (260449)
- Jakub Wierzchowiec (252738)
- Adam Stypczyc (259273)

---

## **Cel projektu**

Naszym zadaniem było opracowanie metod steganografii tekstu. Postanowiliśmy skupić się na różnych rodzajach tej dziedziny.

Poniższy dokument zawiera opracowanie następujących metod steganograficznych:

1. **Użycie specjalnych znaków o zerowej szerokości do ukrywania wiadomości**
2. **Zakodowywanie wiadomości za pomocą losowych imion i nazwisk**
3. **Steganografia synonimami**
4. **Steganografia homoglifami**


# 1. Użycie specjalnych znaków o zerowej szerokości do ukrywania wiadomości

## Funkcja ukrywająca wiadomość przy użyciu znaków o zerowej szerokości

### **Parametry**
- `plain_text` *(str)*: Tekst, do którego chcemy dodać ukrytą wiadomość.
- `secret_message` *(str)*: Wiadomość do ukrycia.

### **Zwraca**
- `encoded_text` *(str)*: Tekst oryginalny z dodaną zaszyfrowaną wiadomością.


In [1]:
def encrypt(plain_text, secret_message):
    # Kodowanie ukrytej wiadomości na postać binarną
    binary_message = ''.join(format(ord(char), '08b') for char in secret_message)
    zero_width_encoding = binary_message.replace('0', '\u200B').replace('1', '\u200C')

    # Łączenie tekstu oryginalnego z zakodowaną wiadomością
    encoded_text = plain_text + '\u200D' + zero_width_encoding + '\u200D'
    return encoded_text

## Funkcja odszyfrowująca ukrytą wiadomość z zakodowanego tekstu

### **Parametry**
- `encoded_text` *(str)*: Tekst z ukrytą wiadomością.

### **Zwraca**
- `plain_text` *(str)*: Tekst oryginalny bez ukrytej wiadomości.
- `secret_message` *(str)*: Ukryta wiadomość.

In [2]:
def decrypt(encoded_text):
    # Sprawdzanie, czy w tekście znajduje się ukryta wiadomość
    if '\u200D' not in encoded_text:
        return "Brak ukrytej wiadomości.", encoded_text

    # Rozdzielanie tekstu oryginalnego i zakodowanej wiadomości
    plain_text, encoded_message = encoded_text.split('\u200D', 1)
    encoded_message = encoded_message.strip('\u200D')

    # Konwersja znaków o zerowej szerokości z powrotem do postaci binarnej
    binary_message = encoded_message.replace('\u200B', '0').replace('\u200C', '1')

    # Konwersja binarnej postaci na znaki, aby uzyskać ukrytą wiadomość
    secret_message = ''.join(chr(int(binary_message[i:i + 8], 2)) for i in range(0, len(binary_message), 8))

    return plain_text, secret_message

### Przykład użycia:

In [3]:
plain_text = "Hello, jak się masz?"
secret_message = "Ukryta123"

# Szyfrowanie
encoded_text = encrypt(plain_text, secret_message)
print("Zakodowany tekst:", encoded_text)

# Odszyfrowanie
original_text, decoded_message = decrypt(encoded_text)
print("Tekst oryginalny:", original_text)
print("Odszyfrowana wiadomość:", decoded_message)

Zakodowany tekst: Hello, jak się masz?‍​‌​‌​‌​‌​‌‌​‌​‌‌​‌‌‌​​‌​​‌‌‌‌​​‌​‌‌‌​‌​​​‌‌​​​​‌​​‌‌​​​‌​​‌‌​​‌​​​‌‌​​‌‌‍
Tekst oryginalny: Hello, jak się masz?
Odszyfrowana wiadomość: Ukryta123


# 2. Zakodowywanie wiadomości za pomocą losowych imion i nazwisk

Steganografia używająca imion nazwisk i do kodowania ukrytej wiadomości. Jest trudna do zauważenia ponieważ zakodowany tekst wygląda jak zwyczajna lista imion i nazwisk danego departamentu. Wiadomość jest zamieniana na ciąg bitów, które następnie są zakodowywane za pomocą imion i nazwisk. Jeżeli bit jest zerem to wybieramy imię lub nazwisko zaczynajace się samogłoską. Jeżeli bit jest jedynką to zamieniamy go na imię lub nazwisko zaczynające się spółgłoska.

In [1]:
from faker import Faker

# biblioteka Faker służy do generowania losowych imion i nazwisk o określonych cechach
fake = Faker()


# Funkcja sprawdzająca pierwszą literę wyrazu i zwracjąca 0 jeżeli litera jest samogłoską i
# 1 jeśli jest spółgłoską
def letter_to_bit(letter):
    vowels = "AEIOU"
    return '0' if letter.upper() in vowels else '1'

In [4]:
# Funkcja zakodowywująca pojedyńczy bajt jako departament
def generate_department(byte):
    department = []
    bits = f"{byte:08b}"  # Konwersja bajtu do 8-bitowego stringa

    for i in range(0, 8, 2):  # Każda osoba moze zakodować 2 bity
        # Generuje losowe imiona dopóki nie otrzymamy, imienia z porządanymi pierwszymi literami

        while True:
            name = fake.first_name()
            surname = fake.last_name()
            name_bit = letter_to_bit(name[0])
            surname_bit = letter_to_bit(surname[0])

            if name_bit == bits[i] and surname_bit == bits[i + 1]:
                department.append((name, surname))
                break

    return department


# Funkcja zakodowywujące wiadomość w departamenty
def encode_message(message):
    byte_array = message.encode('utf-8')
    departments = []

    for byte in byte_array:
        department = generate_department(byte)
        departments.append(department)

    return departments


# Funkcja odkodowywująca wiadomość z departamentów
def decode_message(departments):
    bytes_list = []

    for department in departments:
        bits = ""
        for name, surname in department:
            # Poberamy pierwsze litery z imienia i nazwiska
            name_bit = letter_to_bit(name[0])
            surname_bit = letter_to_bit(surname[0])
            bits += name_bit + surname_bit

        # Konwertujemy 1 literę, czyli 8 bitów w bajt i dodajemy do listy
        byte = int(bits, 2)
        bytes_list.append(byte)

    # Konwerujemy listę bloków bajtów w oryginalny string
    decoded_message = bytes(bytes_list).decode('utf-8')
    return decoded_message

## Przykład użycia

In [5]:
secret_message = "Hello"
encoded_departments = encode_message(secret_message)

print("Encoded Departments:")
for i, department in enumerate(encoded_departments):
    print(f"Department {i + 1}:")
    for name, surname in department:
        print(f"  - {name} {surname}")

# Dekodowanie wiadomości
decoded_message = decode_message(encoded_departments)
print("\nDecoded Message:", decoded_message)

Encoded Departments:
Department 1:
  - Angela Martin
  - Ashley Arias
  - Jesse Andrews
  - Aaron Alvarez
Department 2:
  - Amanda Gonzalez
  - Keith Allen
  - Amanda Strickland
  - Amy Murillo
Department 3:
  - Autumn Collins
  - Megan Acevedo
  - Daniel Williams
  - Albert Alexander
Department 4:
  - Amy Harvey
  - Jesse Eaton
  - Dennis Barrett
  - Adriana Alvarez
Department 5:
  - Andrew Rose
  - Thomas Anderson
  - Kelly Daniel
  - Ryan Fowler

Decoded Message: Hello
